# Kachra Rani 
## Team:
### 1. Syed Hassan Haider
### 2. Inara Naseem G H Chagani 
### 3. Muhammad Shahrom Ali 

In [78]:
# Input: Image 
# Output : Label for that Image 

# Steps: 
# 1. Loading Image and Preprocessing 
# 2. Defining Model Architecture 
# 3. Training the Model 
# 4. Estimation of performance

###### Importing and Housekeeping stuff

In [1]:
#imports
import numpy as np 

import pickle

import cv2

from sklearn import svm
from skimage import color

from skimage.feature import hog
from sklearn.metrics import classification_report,accuracy_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.style.use('fivethirtyeight')

In [2]:
#FILENAMES FOR EXPORTED DATA 

FEATURE_DATA = 'extracted_features'
RAW_DATASET = 'KR_dataset'
KR_MODEL = 'KR_model'

# Supporting information defined 
dataset_info = {
    # FORMAT::garbage_type : (no_of_imgs, label_value)
    'glass' : (501, 0),
    'plastic' : (482, 1),
    'cardboard' : (403, 2),
    'paper' : (594, 3),
    'metal' : (410, 4),
    'trash' : (137, 5)
}

#Label_value (int) : garbage_type (string)
labels = {dataset_info[i][1] : i for i in dataset_info}

###### Helper Functions 

In [3]:
#export files
def export(filename, to_export):
    np.save(filename + ".npy", to_export)

In [4]:
def load(filename):
    test_frame = np.load(filename + ".npy")

In [5]:
def preprocess_img(img):
    
    scale_percent = 50 # percent of original size
    
    #Resize the image for less pixels 
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height) 
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

    #convert image to grayscale 
    img = color.rgb2gray(img)
    
    return img

###### Load Data

In [6]:
KR_dataset = list()

for garbage_type in dataset_info:
    
    img_count = dataset_info[garbage_type][0]
    garbage_label = dataset_info[garbage_type][1]
    
    #LOAD IMAGES OF SELECTED GARBAGE TYPE
    for i in range(1,  img_count + 1):
        
        img_path = './Dataset/' + garbage_type + "/" + garbage_type + str(i) + '.jpg'
        
        img = preprocess_img( mpimg.imread( img_path ) ) 
        
        KR_dataset.append((img, garbage_label))
        
#Loaded Dataset
KR_dataset = np.array(KR_dataset, dtype=object)

In [7]:
#export RAW Dataset
export(RAW_DATASET, KR_dataset)

In [8]:
#DATASET SHUFFLED
np.random.shuffle(KR_dataset)

###### Feature Extraction 

In [9]:
def feature_extraction(img):
    ppc = 16 #PIXELS PER CEL 
    return hog(img, 
               orientations=8,
               pixels_per_cell=(ppc,ppc),
               cells_per_block=(4, 4),
               block_norm= 'L2',
               visualize=True)

In [10]:
#Feature Extraction using Histogram of Gradient
hog_images = []
hog_features = []

for image in KR_dataset:
    
    feat_desc, hog_image = feature_extraction(image[0])

    hog_images.append(hog_image)
    hog_features.append(feat_desc)

In [11]:
hog_features = np.array(hog_features)

labels = np.array( [img[1] for img in KR_dataset] ).reshape(len(KR_dataset), 1) #make collumn vector

# each row corresponds to hog_features of some img i
# and at the end is the label of that image 
data_frame = np.hstack((hog_features, labels))

In [12]:
#Export Extracted Feature 
export(FEATURE_DATA, data_frame)

###### Splitting Data

In [13]:
percentage = 80
partition = int(len(hog_features)*percentage/100)

In [14]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

###### Generating Model

In [15]:
clf = svm.SVC()
clf.fit(x_train,y_train)

SVC()

In [20]:
#Export model
export(KR_MODEL, clf)

###### Predictions and Model Evaluation

In [21]:
y_pred = clf.predict(x_test)

In [46]:
for i in range(len(x_test)):
    print("The predicted value is:", y_pred[i])
    print("The actual value is:", y_test[i])    

The predicted value is: 3.0
The actual value is: 2.0
The predicted value is: 3.0
The actual value is: 4.0
The predicted value is: 0.0
The actual value is: 2.0
The predicted value is: 0.0
The actual value is: 4.0
The predicted value is: 1.0
The actual value is: 1.0
The predicted value is: 0.0
The actual value is: 5.0
The predicted value is: 4.0
The actual value is: 0.0
The predicted value is: 2.0
The actual value is: 2.0
The predicted value is: 0.0
The actual value is: 0.0
The predicted value is: 3.0
The actual value is: 3.0
The predicted value is: 3.0
The actual value is: 3.0
The predicted value is: 1.0
The actual value is: 1.0
The predicted value is: 0.0
The actual value is: 1.0
The predicted value is: 3.0
The actual value is: 3.0
The predicted value is: 0.0
The actual value is: 1.0
The predicted value is: 3.0
The actual value is: 4.0
The predicted value is: 4.0
The actual value is: 0.0
The predicted value is: 3.0
The actual value is: 4.0
The predicted value is: 1.0
The actual value i

In [22]:
print("Accuracy: " + str(accuracy_score(y_test, y_pred) * 100) + "\%")
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 57.509881422924906\%


              precision    recall  f1-score   support

         0.0       0.48      0.67      0.56        87
         1.0       0.55      0.49      0.52       104
         2.0       0.72      0.68      0.70        88
         3.0       0.58      0.72      0.65       116
         4.0       0.57      0.41      0.48        85
         5.0       1.00      0.12      0.21        26

    accuracy                           0.58       506
   macro avg       0.65      0.52      0.52       506
weighted avg       0.60      0.58      0.56       506

